In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import os, sys, itertools, csv
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.alemutdf import get_all_sample_mut_df, get_multi_exp_max_freq_mut_df
from util.mut import is_coding_mut
from util.metadata import get_condition_val_dict, get_condition_field_val_set
from util.genome import get_K12_pos_from_BOP27

In [2]:
muts_df = get_all_sample_mut_df("./data/muts/pub_growth/")
# unpublished_muts_df = get_all_sample_mut_df("./data/muts/unpublished/")  # I've currently placed GYD into the public mutation set.
# muts_df = muts_df.append(unpublished_muts_df)
display(set(muts_df["exp"]))

  0%|          | 0/20 [00:00<?, ?it/s]

./data/muts/pub_growth//MG1655_untreated_mut.csv
./data/muts/pub_growth//42C.csv
./data/muts/pub_growth//Iron-ALE_mut.csv
./data/muts/pub_growth//MG1655-M9-NC_000913_3gb-stationary-37-monomethyl_succinate2_mut.csv
./data/muts/pub_growth//gallium-nitrate_mut.csv
./data/muts/pub_growth//TOL adipic acid_mut.csv


  0%|          | 0/67 [00:00<?, ?it/s]

./data/muts/pub_growth//MG1655-blaTEM1_untreated_mut.csv
./data/muts/pub_growth//tpiKO_mut.csv
./data/muts/pub_growth//TOL n-butanol_mut.csv
./data/muts/pub_growth//pvdJ_KO_P_aeruginosa_spent_medium_mut.csv
./data/muts/pub_growth//tpiPAE_mut.csv


  0%|          | 0/24 [00:00<?, ?it/s]

...

 36%|███▋      | 24/66 [00:00<00:00, 239.80it/s]

./data/muts/pub_growth//MG1655_anchestor_mut.csv
./data/muts/pub_growth//pgiKO_mut.csv
./data/muts/pub_growth//pgi_mut.csv


100%|██████████| 10/10 [00:00<00:00, 238.94it/s]

./data/muts/pub_growth//MG1655-pBGT_untreated_mut.csv
./data/muts/pub_growth//C13.csv
./data/muts/pub_growth//MG1655-blaTEM1_ceftazidime_mut.csv
./data/muts/pub_growth//gnd_mut.csv


{'42C',
 'C13',
 'EG_pCA24N-fucO',
 'GLU',
 'GYD',
 'ILT_MG1655',
 'Iron-ALE',
 'JCL260__isobutanol_production_host-M9-MG1655gbk',
 'MG1655-M9-NC_000913_3gb-stationary-37-D-2-deoxyribose2',
 'MG1655-M9-NC_000913_3gb-stationary-37-D-arabinose2',
 'MG1655-M9-NC_000913_3gb-stationary-37-D-lyxose2',
 'MG1655-M9-NC_000913_3gb-stationary-37-m-tartrate2',
 'MG1655-M9-NC_000913_3gb-stationary-37-monomethyl_succinate2',
 'MG1655-blaTEM1_ceftazidime',
 'MG1655-blaTEM1_untreated',
 'MG1655-pBGT_ceftazidime',
 'MG1655-pBGT_untreated',
 'MG1655_anchestor',
 'MG1655_ceftazidime',
 'MG1655_chloramphenicol',
 'MG1655_doxycycline',
 'MG1655_trimethoprim',
 'MG1655_untreated',
 'OxidizeME_PQ3',
 'PGI',
 'SER',
 'SSW_AC',
 'SSW_GLU_AC',
 'SSW_GLU_GLY',
 'SSW_GLU_XYL',
 'SSW_GLY',
 'SSW_XYL',
 'TOL_2,3-butanediol',
 'TOL_adipic_acid',
 'TOL_coumaric_acid',
 'TOL_glutaric_acid',
 'TOL_hexamethylenediamine',
 'TOL_hexanoic_acid',
 'TOL_isobutyric_acid',
 'TOL_n-butanol',
 'TOL_octanoic_acid',
 'TOL_propaned

## remove problem and starting strain mutations

In [3]:
# Different experiments have different strings for position (some with commas, some without), therefore going ahead and changing them all to integers
muts_df.Position = muts_df.Position.apply(lambda x: int(str(x).replace(",","")))
muts_df.Position = muts_df.Position.astype(int)

In [4]:
muts_df = muts_df[~(
# For whatever reason, the gene annotations can change between version of breseq or runs
#     ((muts_df.Gene == "gatC")
#      | (muts_df.Gene == "gatC, gatC"))
    ((muts_df.Position == 2173361)
       | (muts_df.Position == 2173363)
       | (muts_df.Position == 2173364))
    & ((muts_df["Sequence Change"] == "Δ2 bp")
       | (muts_df["Sequence Change"] == "Δ1 bp"))
)]

muts_df = muts_df[~(
#     (muts_df.Gene == "[crl]")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 257908)
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "insB1, insA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1978503)
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "ychE, oppA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1299499)
    & (muts_df["Sequence Change"] == "Δ1,199 bp"))]

muts_df = muts_df[~(
# For whatever reason, the gene annotations can change between version of breseq or runs
#     ((muts_df.Gene == "glpR")
#      | (muts_df.Gene == "glpR, glpR")
#      | (muts_df.Gene == "glpR/glpR"))
    (muts_df.Position == 3560455)
    & (muts_df["Sequence Change"] == "+G"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "gltP, yjcO")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 4296381)
    & (muts_df["Sequence Change"] == "+GC"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "gltP, yjcO")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 4296380)
    & (muts_df["Sequence Change"] == "+CG"))]

muts_df = muts_df[~(
#     (muts_df.Gene == "insB1,insA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1978503)
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
#     ((muts_df.Gene == "insA/uspC") | (muts_df.Gene == "insA, uspC"))  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.Position == 1979486)
    & (muts_df["Sequence Change"] == "IS5 (+) +4 bp"))]


muts_df = muts_df[~(
#     (muts_df.Gene == "corA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.exp != "GLU")  # This is the ALE that originated the corA mutation that is found in all CCK samples, therefore should track
    & (muts_df.Position == 4001645)
    & (muts_df["Sequence Change"] == "Δ5 bp"))]


muts_df = muts_df[~(
#     (muts_df.Gene == "wcaA")  # For whatever reason, the gene annotations can change between version of breseq or runs
    (muts_df.exp != "GLU")  # This is the ALE that originated the corA mutation that is found in all CCK samples, therefore should track
    & (muts_df.Position == 2132787)
    & (muts_df["Details"] == "I204S (ATC→AGC)"))]


p = {4293212,
 4293403,
 4293430,
 4296060,
 4296154,
 4296189,
 4296190,
 4296191,
 4296267,
 4296268,
 4296286,
 4296380,
 4296382}
ch = {'+C', '+G', 'A→C', 'A→G', 'A→T', 'C→A', 'C→T', 'T→C'}
muts_df = muts_df[~(
#     (muts_df.Gene == "gltP, yjcO")
    (muts_df["Position"].isin(p))
    & (muts_df["Sequence Change"].isin(ch))
       )]

display(len(muts_df), muts_df.head())

17509

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,Mut ID
6,MG1655_untreated,1,16,0,1,1.00,1397381,DEL,"Δ13,756 bp",,"[ynaJ],uspE,fnr,ogt,abgT,abgB,abgA,abgR,mcaS,s...",NC_000913,1608368.0
14,MG1655_untreated,1,16,0,1,0.61,4542237,MOB,IS1 (+) +9 bp,coding (201‑209/597 nt),fimE,NC_000913,1608475.0
6,MG1655_untreated,2,16,0,1,1.00,1397381,DEL,"Δ13,756 bp",,"[ynaJ],uspE,fnr,ogt,abgT,abgB,abgA,abgR,mcaS,s...",NC_000913,1608368.0
11,MG1655_untreated,2,16,0,1,0.22,4542456,MOB,IS1 (+) +9 bp,coding (420‑428/597 nt),fimE,NC_000913,1608472.0
12,MG1655_untreated,2,16,0,1,0.21,4542831,MOB,IS5 (+) +4 bp,intergenic (+198/‑281),"fimE, fimA",NC_000913,1608473.0


In [5]:
reduced_ecoli_ale = 'application_of_ALE_to_re-optimize_growth_performance_of_genome-reduced_strain_MS56-M9-MG1655gbk'
MG1655_pBGT_untreated = "MG1655-pBGT_untreated"
MG1655_untreated = "MG1655_untreated"
MG1655_blaTEM1_untreated = "MG1655-blaTEM1_untreated"
muts_to_remove = [
    {"experiment":MG1655_blaTEM1_untreated,"position":1397381,"sequence change":"Δ13,756 bp"},
    {"experiment":MG1655_untreated,"position":1397381,"sequence change":"Δ13,756 bp"},
    {"experiment":MG1655_pBGT_untreated,"position":1397381,"sequence change":"Δ13,756 bp"},
    {"experiment":reduced_ecoli_ale,"position":389898,"sequence change":"Δ9,908 bp"},
    {"experiment":reduced_ecoli_ale,"position":1388889,"sequence change":"Δ9,734 bp"},
    {"experiment":reduced_ecoli_ale,"position":2466545,"sequence change":"Δ9,634 bp"},
    {"experiment":reduced_ecoli_ale,"position":729365,"sequence change":"Δ9,598 bp"},
    {"experiment":reduced_ecoli_ale,"position":3815859,"sequence change":"Δ82 bp"},
    {"experiment":reduced_ecoli_ale,"position":1400325,"sequence change":"Δ81,931 bp"},
    {"experiment":reduced_ecoli_ale,"position":380112,"sequence change":"Δ8,537 bp"},
    {"experiment":reduced_ecoli_ale,"position":2509631,"sequence change":"Δ8,319 bp"},
    {"experiment":reduced_ecoli_ale,"position":3761993,"sequence change":"Δ8,250 bp"},
    {"experiment":reduced_ecoli_ale,"position":257908,"sequence change":"Δ776 bp"},
    {"experiment":reduced_ecoli_ale,"position":1978503,"sequence change":"Δ776 bp"},
    {"experiment":reduced_ecoli_ale,"position":1040502,"sequence change":"Δ73,763 bp"},
    {"experiment":reduced_ecoli_ale,"position":3090347,"sequence change":"Δ708 bp"},
    {"experiment":reduced_ecoli_ale,"position":675552,"sequence change":"Δ7,819 bp"},
    {"experiment":reduced_ecoli_ale,"position":4400289,"sequence change":"Δ7 bp"},
    {"experiment":reduced_ecoli_ale,"position":819776,"sequence change":"Δ62,977 bp"},
    {"experiment":reduced_ecoli_ale,"position":263514,"sequence change":"Δ61,897 bp"},
    {"experiment":reduced_ecoli_ale,"position":3184782,"sequence change":"Δ6,916 bp"},
    {"experiment":reduced_ecoli_ale,"position":997868,"sequence change":"Δ6,818 bp"},
    {"experiment":reduced_ecoli_ale,"position":2558699,"sequence change":"Δ6,790 bp"},
    {"experiment":reduced_ecoli_ale,"position":3618992,"sequence change":"Δ6,688 bp"},
    {"experiment":reduced_ecoli_ale,"position":4496676,"sequence change":"Δ53,036 bp"},
    {"experiment":reduced_ecoli_ale,"position":167484,"sequence change":"Δ5,964 bp"},
    {"experiment":reduced_ecoli_ale,"position":603467,"sequence change":"Δ5,885 bp"},
    {"experiment":reduced_ecoli_ale,"position":3362162,"sequence change":"Δ5,481 bp"},
    {"experiment":reduced_ecoli_ale,"position":765208,"sequence change":"Δ5,405 bp"},
    {"experiment":reduced_ecoli_ale,"position":15389,"sequence change":"Δ5,175 bp"},
    {"experiment":reduced_ecoli_ale,"position":1509562,"sequence change":"Δ46,201 bp"},
    {"experiment":reduced_ecoli_ale,"position":332366,"sequence change":"Δ44,951 bp"},
    {"experiment":reduced_ecoli_ale,"position":4555491,"sequence change":"Δ41,523 bp"},
    {"experiment":reduced_ecoli_ale,"position":508873,"sequence change":"Δ40,744 bp"},
    {"experiment":reduced_ecoli_ale,"position":1223090,"sequence change":"Δ4 bp"},
    {"experiment":reduced_ecoli_ale,"position":2101397,"sequence change":"Δ36,321 bp"},
    {"experiment":reduced_ecoli_ale,"position":2756160,"sequence change":"Δ35,091 bp"},
    {"experiment":reduced_ecoli_ale,"position":2204350,"sequence change":"Δ32,368 bp"},
    {"experiment":reduced_ecoli_ale,"position":2994992,"sequence change":"Δ3,877 bp"},
    {"experiment":reduced_ecoli_ale,"position":2286401,"sequence change":"Δ3,780 bp"},
    {"experiment":reduced_ecoli_ale,"position":1997112,"sequence change":"Δ26,567 bp"},
    {"experiment":reduced_ecoli_ale,"position":1197137,"sequence change":"Δ25,940 bp"},
    {"experiment":reduced_ecoli_ale,"position":4299628,"sequence change":"Δ25,538 bp"},
    {"experiment":reduced_ecoli_ale,"position":1627518,"sequence change":"Δ25,244 bp"},
    {"experiment":reduced_ecoli_ale,"position":565055,"sequence change":"Δ21,054 bp"},
    {"experiment":reduced_ecoli_ale,"position":2846301,"sequence change":"Δ20,968 bp"},
    {"experiment":reduced_ecoli_ale,"position":3651293,"sequence change":"Δ2,422 bp"},
    {"experiment":reduced_ecoli_ale,"position":2173363,"sequence change":"Δ2 bp"},
    {"experiment":reduced_ecoli_ale,"position":3888432,"sequence change":"Δ18,167 bp"},
    {"experiment":reduced_ecoli_ale,"position":1962566,"sequence change":"Δ16,764 bp"},
    {"experiment":reduced_ecoli_ale,"position":2320144,"sequence change":"Δ16,489 bp"},
    {"experiment":reduced_ecoli_ale,"position":3453928,"sequence change":"Δ15,926 bp"},
    {"experiment":reduced_ecoli_ale,"position":2601201,"sequence change":"Δ13,595 bp"},
    {"experiment":reduced_ecoli_ale,"position":2165151,"sequence change":"Δ12,060 bp"},
    {"experiment":reduced_ecoli_ale,"position":1129397,"sequence change":"Δ11,591 bp"},
    {"experiment":reduced_ecoli_ale,"position":3593299,"sequence change":"Δ10 bp"},
    {"experiment":reduced_ecoli_ale,"position":3720634,"sequence change":"Δ1,442 bp"},
    {"experiment":reduced_ecoli_ale,"position":1872030,"sequence change":"Δ1,437 bp"},
    {"experiment":reduced_ecoli_ale,"position":1299499,"sequence change":"Δ1,199 bp"},
    {"experiment":reduced_ecoli_ale,"position":687860,"sequence change":"Δ1,185 bp"},
    {"experiment":reduced_ecoli_ale,"position":3581139,"sequence change":"3915 bp→6 bp"},
    {"experiment":reduced_ecoli_ale,"position":1197137,"sequence change":"25956 bp→12 bp"},
    {"experiment":reduced_ecoli_ale,"position":3110680,"sequence change":"25702 bp→GA"},
]

In [6]:
for mut in muts_to_remove:
    muts_df = muts_df[~(
        (muts_df["exp"] != mut["experiment"])
        & (muts_df["Position"] == mut["position"])
        & (muts_df["Sequence Change"] == mut["sequence change"])
    )]
display(len(muts_df), muts_df.head())

17184

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,Mut ID
14,MG1655_untreated,1,16,0,1,0.61,4542237,MOB,IS1 (+) +9 bp,coding (201‑209/597 nt),fimE,NC_000913,1608475.0
11,MG1655_untreated,2,16,0,1,0.22,4542456,MOB,IS1 (+) +9 bp,coding (420‑428/597 nt),fimE,NC_000913,1608472.0
12,MG1655_untreated,2,16,0,1,0.21,4542831,MOB,IS5 (+) +4 bp,intergenic (+198/‑281),"fimE, fimA",NC_000913,1608473.0
8,MG1655_untreated,3,16,0,1,0.49,4542191,SNP,A→C,D52A (GAC→GCC),fimE,NC_000913,1608466.0
9,MG1655_untreated,3,16,0,1,0.33,4542298,MOB,IS1 (+) +8 bp,coding (262‑269/597 nt),fimE,NC_000913,1608467.0


## Remove experiment specific starting strain mutations

In [7]:
# Filter the following mutations out if they come from specific experiments.
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="cyoB") & (muts_df["Details"]=="W190* (TGG→TAG)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="eno") & (muts_df["Details"]=="L193Q (CTG→CAG)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="chbF") & (muts_df["Details"]=="V133E (GTA→GAA)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="fumC") & (muts_df["Details"]=="G354G (GGT→GGC)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="yegE") & (muts_df["Details"]=="S683Y (TCC→TAC)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="rcsC") & (muts_df["Details"]=="L594F (TTG→TTC)")))]


muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="fumC") & (muts_df["Details"]=="G354G (GGT→GGC)")))]
muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="yegE") & (muts_df["Details"]=="S683Y (TCC→TAC)")))]
muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="rcsC") & (muts_df["Details"]=="L594F (TTG→TTC)")))]


cck_exp_list = [
    "Central_carbon_knockout_tpiA",
    "Central_carbon_knockout_PTS",
    "Central_carbon_knockout_pgi",
    "Central_carbon_knockout_sdh",
    "Central_carbon_knockout_gnd",
    "Central_carbon_knockout_glucose_evolution"
]

muts_df = muts_df[~(
    (muts_df.exp.isin(cck_exp_list))
    & (
        (muts_df["Gene"]=="wcaA")
        | (muts_df["Gene"]=="corA")
    )
)]  # The details of the mutations to filter out should probably include the sequence change.

### Removing MG1655_anchestor mutations

In [8]:
# to use with filtering out mutations 
muts_df = muts_df.reset_index(drop=True)

In [9]:
# Not many. Going to manual remove (see cell below)
# This is for the Ecoli_resistance_caz
# Sebastian often uses the name "MG1655_anchestor" 
MG1655_anchestor = muts_df[muts_df.exp=="MG1655_anchestor"].copy()
MG1655_anchestor

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,Mut ID
16116,MG1655_anchestor,0,0,1,1,1.0,4390754,SNP,G→T,A252A (GCC→GCA),rsgA,NC_000913,1608306.0
16117,MG1655_anchestor,0,0,1,1,1.0,803662,SNP,C→A,L54I (CTT→ATT),ybhJ,NC_000913,1608301.0
16118,MG1655_anchestor,0,0,1,1,1.0,1905761,SNP,G→A,G25D (GGT→GAT),mntP,NC_000913,1608303.0


In [10]:
print(len(muts_df))
muts_df = muts_df[~(muts_df.index.isin(list(MG1655_anchestor.index)))]
print(len(muts_df))

16980
16977


In [11]:
muts_df = muts_df[(~((muts_df.exp=="MG1655_chloramphenicol") & (muts_df["Position"]==4390754) & (muts_df["Details"]=="A252A (GCC→GCA)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_chloramphenicol") & (muts_df["Position"]==803662) & (muts_df["Details"]=="L54I (CTT→ATT)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_chloramphenicol") & (muts_df["Position"]==1905761) & (muts_df["Details"]=="G25D (GGT→GAT)")))]

muts_df = muts_df[(~((muts_df.exp=="MG1655_doxycycline") & (muts_df["Position"]==4390754) & (muts_df["Details"]=="A252A (GCC→GCA)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_doxycycline") & (muts_df["Position"]==803662) & (muts_df["Details"]=="L54I (CTT→ATT)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_doxycycline") & (muts_df["Position"]==1905761) & (muts_df["Details"]=="G25D (GGT→GAT)")))]

muts_df = muts_df[(~((muts_df.exp=="MG1655_trimethoprim") & (muts_df["Position"]==4390754) & (muts_df["Details"]=="A252A (GCC→GCA)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_trimethoprim") & (muts_df["Position"]==803662) & (muts_df["Details"]=="L54I (CTT→ATT)")))]
muts_df = muts_df[(~((muts_df.exp=="MG1655_trimethoprim") & (muts_df["Position"]==1905761) & (muts_df["Details"]=="G25D (GGT→GAT)")))]
print(len(muts_df))

16920


Though the other Ecoli_resistance_caz experiments have "anchestor" data on the aledb.ucsd.edu server, their mutations aren't uploaded, therefore not going to filter anything from them at this point.

### Starting strain mutations according to samples within experiments with starting strain AFIRs

In [12]:
# to use with filtering out mutations 
muts_df = muts_df.reset_index(drop=True)

In [13]:
def get_sample_name(ale, flask, isolate, tech_rep):
    sample_name = str(int(ale)) + " " + str(int(flask)) + " " + str(int(isolate)) + " " + str(int(tech_rep))
    return sample_name

muts_df["sample"] = muts_df.apply(lambda r: get_sample_name(r.ale, r.flask, r.isolate, r.tech_rep), axis=1)
muts_df

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,Mut ID,sample
0,MG1655_untreated,1,16,0,1,0.61,4542237,MOB,IS1 (+) +9 bp,coding (201‑209/597 nt),fimE,NC_000913,1608475.0,1 16 0 1
1,MG1655_untreated,2,16,0,1,0.22,4542456,MOB,IS1 (+) +9 bp,coding (420‑428/597 nt),fimE,NC_000913,1608472.0,2 16 0 1
2,MG1655_untreated,2,16,0,1,0.21,4542831,MOB,IS5 (+) +4 bp,intergenic (+198/‑281),"fimE, fimA",NC_000913,1608473.0,2 16 0 1
3,MG1655_untreated,3,16,0,1,0.49,4542191,SNP,A→C,D52A (GAC→GCC),fimE,NC_000913,1608466.0,3 16 0 1
4,MG1655_untreated,3,16,0,1,0.33,4542298,MOB,IS1 (+) +8 bp,coding (262‑269/597 nt),fimE,NC_000913,1608467.0,3 16 0 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16915,gnd,19,171,2,1,1.00,1697375,SNP,T→C,Q222Q (CAA→CAG),hdhA,NC_000913,799561.0,19 171 2 1
16916,gnd,19,171,2,1,1.00,3951735,DEL,Δ326 bp,pseudogene (194‑519/663 nt),ilvG,NC_000913,1114004.0,19 171 2 1
16917,gnd,19,171,2,1,1.00,3700725,SNP,A→T,I55F (ATC→TTC),yhjV,NC_000913,1566613.0,19 171 2 1
16918,gnd,19,171,2,1,1.00,3968459,SNP,C→T,Q182* (CAA→TAA),wecA,NC_000913,1566614.0,19 171 2 1


In [14]:
# Remove starting strain mutations from all experiments according to each experiment's starting strain
muts_df = muts_df.copy()
starting_strains = muts_df[muts_df["sample"]=="0 0 1 1"]
muts_df = muts_df[muts_df["sample"]!="0 0 1 1"]  # remove starting strain samples
for e, exp_starting_strain_muts in starting_strains.groupby("exp"):
    rows_to_filter = (
        muts_df["exp"].isin(exp_starting_strain_muts["exp"])
        & muts_df["Position"].isin(exp_starting_strain_muts["Position"])
        & muts_df["Sequence Change"].isin(exp_starting_strain_muts["Sequence Change"])
        & muts_df["Details"].isin(exp_starting_strain_muts["Details"])
    )
    muts_df.drop(muts_df[rows_to_filter].index, inplace=True)  # have to ensure that muts_df indeces are unique

###  Check that no weird mutations references are included in dataset, since they won't match to RegulonDB's data

In [15]:
for ref_seq, df in muts_df.groupby("Reference Seq"):
    print(ref_seq, df.exp.unique())

CP009273 ['TOL_isobutyric_acid']
NC_000913 ['MG1655_untreated' 'Iron-ALE'
 'MG1655-M9-NC_000913_3gb-stationary-37-monomethyl_succinate2'
 'gallium-nitrate' 'TOL_adipic_acid' 'MG1655-blaTEM1_untreated' 'tpiKO'
 'TOL_n-butanol' 'pvdJ_KO_P_aeruginosa_spent_medium' 'TOL_octanoic_acid'
 'TOL_glutaric_acid'
 'MG1655-M9-NC_000913_3gb-stationary-37-D-2-deoxyribose2'
 'MG1655-M9-NC_000913_3gb-stationary-37-D-lyxose2' 'TOL_putrescine'
 'WT_P_aeruginosa_spent_medium' 'EG_pCA24N-fucO'
 'TOL_hexamethylenediamine' 'MG1655_doxycycline' 'evo'
 'JCL260__isobutanol_production_host-M9-MG1655gbk'
 'MG1655-M9-NC_000913_3gb-stationary-37-D-arabinose2' 'TOL_propanediol'
 'TOL_2,3-butanediol' 'sdh' 'GLU' 'MG1655_trimethoprim'
 'MG1655-pBGT_ceftazidime' 'TOL_hexanoic_acid' 'wt'
 'application_of_ALE_to_re-optimize_growth_performance_of_genome-reduced_strain_MS56-M9-MG1655gbk'
 'pyocyanin_supplemented_medium' 'MG1655_ceftazidime' 'OxidizeME_PQ3'
 'tpi' 'MG1655_chloramphenicol' 'TOL_coumaric_acid' 'TOL_isobutyric

According the the prevous results, TOL_isobutyric_acid has one weird mutation. Going to remove it.

In [16]:
muts_df = muts_df[muts_df["Reference Seq"] != "CP009273"]
muts_df["Reference Seq"].unique()

array(['NC_000913', nan, 'NC_000913_3_pae_tpiA', 'NC_000913_3_vch_tpiA',
       'NC_000913_3_bme_pgi', 'NC_000913_3_hsa_tpiA',
       'NC_000913_3_hsa_pgi', 'NC_000913_3_vch_pgi',
       'NC_000913_3_pae_pgi', 'NC_000913_3_KO_pgi'], dtype=object)

In [17]:
# Amount of ALEs before removal
print(len(muts_df.apply(lambda r: r.exp + " " + str(r.ale), axis=1).unique()))

# removing hypermutators since they will affect associations

muts_df = muts_df[
    ~(
        (muts_df.exp == "GLU")
        & ((muts_df.ale == 5) | (muts_df.ale == 7))
    )
]

muts_df = muts_df[
    ~(
        (muts_df.exp == "TOL_2,3-butanediol")
        & (muts_df.ale == 3)
    )
]

muts_df = muts_df[
    ~(
        (muts_df.exp == "TOL_adipic_acid")
        & (muts_df.ale == 5)
    )
]

muts_df = muts_df[
    ~(
        (muts_df.exp == "TOL_hexamethylenediamine")
        & (muts_df.ale == 6)
    )
]

muts_df = muts_df[
    ~(
        (muts_df.exp == "TOL_octanoic_acid")
        & (muts_df.ale == 5)
    )
]

muts_df = muts_df[
    ~(
        (muts_df.exp == "TOL_propanediol")
        & ((muts_df.ale == 1) | (muts_df.ale == 2) | (muts_df.ale == 3) | (muts_df.ale == 4) | (muts_df.ale == 5) | (muts_df.ale == 7) | (muts_df.ale == 8))
    )
]

print(len(muts_df.apply(lambda r: r.exp + " " + str(r.ale), axis=1).unique()))

388
375


In [18]:
# removing samples not incuded in GYD MS for AVA MS

muts_df = muts_df[
    ~(
        (muts_df.exp == "GYD")
        & ((muts_df.ale == 35) | (muts_df.ale == 34))
    )
]
print(len(muts_df.apply(lambda r: r.exp + " " + str(r.ale), axis=1).unique()))

373


In [19]:
print(len(muts_df))
muts_df = muts_df[muts_df.presence >= 0.5]
print(len(muts_df))

10008
9514


### Just get ALE-unique mutations 
Since the majority of our comparisons have to do with endpoints, just look at endpoints.  
If not limiting to endpoint, the mutations used in associated mutations to conditions will need filtering.  
This occurs in a later NB of the pipeline.  

In [20]:
muts_df = get_multi_exp_max_freq_mut_df(muts_df, endpoint_flask_only=False)
muts_df.head()

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,Mut ID,sample
9,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,NaN,1 124 1 1
10,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,NaN,1 124 1 1
11,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,NaN,1 124 1 1
12,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,NaN,1 124 1 1
21,42C,2,163,1,1,1.0,149214,SNP,C→T,A130T (GCC→ACC),panB,NaN,NaN,2 163 1 1


In [21]:
# ensure all mutations are using NC_000913 positions.
genome_pos_transform_exp_l = ["ERS",
                              'Central_carbon_knockout_PTS',
                              'Central_carbon_knockout_glucose_evolution',
                              'Central_carbon_knockout_gnd',
                              'Central_carbon_knockout_pgi',
                              'Central_carbon_knockout_sdh',
                              'Central_carbon_knockout_tpiA',]
muts_df.Position = muts_df.apply(lambda row: get_K12_pos_from_BOP27(row.Position) if row.exp in genome_pos_transform_exp_l else row.Position, axis=1)

In [22]:
muts_df["coding"] = muts_df.Details.apply(is_coding_mut)
muts_df["Gene"] = muts_df.apply(lambda row: row["Gene"].replace(", ","/") if not row["coding"] else row["Gene"], axis=1)

muts_df.head()

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,Mut ID,sample,coding
9,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,NaN,1 124 1 1,True
10,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,NaN,1 124 1 1,True
11,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,NaN,1 124 1 1,True
12,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,NaN,1 124 1 1,True
21,42C,2,163,1,1,1.0,149214,SNP,C→T,A130T (GCC→ACC),panB,NaN,NaN,2 163 1 1,True


In [23]:
# This work is also currently duplicated NB4 
muts_df["Gene"] = muts_df["Gene"].apply(lambda a: "rph" if a == "[rph], [rph]" else a)
muts_df["Gene"] = muts_df["Gene"].apply(lambda a: "rph" if a == "[rph],[rph]" else a)
muts_df.head()

,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,Gene,Reference Seq,Mut ID,sample,coding
9,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,NaN,1 124 1 1,True
10,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,NaN,1 124 1 1,True
11,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,NaN,1 124 1 1,True
12,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,NaN,1 124 1 1,True
21,42C,2,163,1,1,1.0,149214,SNP,C→T,A130T (GCC→ACC),panB,NaN,NaN,2 163 1 1,True


In [24]:
# removing unnecessary column
muts_df = muts_df.drop(columns=["Mut ID"])

In [25]:
muts_df.to_pickle("./data/1_df.pkl")

In [26]:
len(muts_df)

6940